In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [3]:
filename = 'stock_data.csv'
df = pd.read_csv(filename, header=0)
df = df[~df['prediction-label'].isna()]
meta_df = df[df['ticker']== 'META']
meta_df = meta_df.drop(columns = ['ticker', 'number-employees', 'date_object' ])
y = meta_df['prediction-label']
X = meta_df.drop(columns = ['prediction-label', 'Date'])
meta_df

,Date,Open,High,Low,Close,Adj Close,Volume,neutral-count-finbert,positive-count-finbert,negative-count-finbert,average-confidence-finbert,average-neutral-score-gemini,average-positive-score-gemini,average-negative-score-gemini,prediction-label
12,2024-11-08,591.489990,593.099976,584.549988,589.340027,589.340027,9254890,1,1,6,0.882399,7.500,3.625,3.250,0.0
24,2024-11-09,591.489990,593.099976,584.549988,589.340027,589.340027,9254890,0,0,8,0.976398,7.250,3.125,1.750,0.0
36,2024-11-11,586.760010,587.000000,576.510071,583.169983,583.169983,10046291,2,0,6,0.954252,7.375,3.750,2.750,0.0
49,2024-11-12,587.544983,599.659973,580.440002,584.820007,584.820007,15694632,1,0,7,0.995730,7.625,3.875,1.250,0.0
62,2024-11-13,582.559998,585.640015,575.179993,580.000000,580.000000,10038132,1,0,7,0.978111,7.875,3.750,2.875,0.0
75,2024-11-14,577.000000,580.760010,573.010010,577.159973,577.159973,10556017,1,0,7,0.977997,8.000,4.375,2.750,1.0
88,2024-11-19,552.000000,559.090027,550.600098,559.025696,559.025696,6085542,2,0,6,0.991861,NaN,NaN,NaN,1.0
99,2024-11-21,569.359985,570.000000,549.049988,563.090027,563.090027,10869427,2,0,6,0.991853,4.500,5.375,4.125,0.0
189,2024-11-22,563.827271,563.827271,554.590027,559.140015,559.140015,9044809,3,0,5,0.924357,4.000,6.250,2.625,0.0
283,2024-11-25,562.979980,572.591492,556.390015,565.109985,565.109985,13460330,4,2,2,0.871256,5.500,5.375,4.000,1.0


In [62]:
filename = 'stock_data.csv'
df = pd.read_csv(filename, header=0)
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'neutral-count-finbert', 'positive-count-finbert',
       'negative-count-finbert', 'average-confidence-finbert',
       'average-neutral-score-gemini', 'average-positive-score-gemini',
       'average-negative-score-gemini', 'prediction-label', 'ticker',
       'number-employees', 'date_object'],
      dtype='object')

In [64]:
df = df.drop(columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'date_object', 'ticker' ]).dropna()

In [66]:
df
#incorporate financial data into this model by adding other columns
#include tickers?
#add sectors instead of tickers - yahoo finance might have sector 
#try prediction-label based on previous day/next day

,neutral-count-finbert,positive-count-finbert,negative-count-finbert,average-confidence-finbert,average-neutral-score-gemini,average-positive-score-gemini,average-negative-score-gemini,prediction-label,number-employees
0,3,1,4,0.995374,6.875,4.375,3.500,1.0,140473.0
1,2,2,4,0.893756,6.875,5.375,3.500,0.0,164000.0
2,5,1,2,0.912988,7.125,5.250,2.375,0.0,6500.0
3,3,3,2,0.926920,6.500,5.750,2.500,0.0,88000.0
4,1,1,6,0.982125,8.250,4.125,1.750,0.0,101200.0
...,...,...,...,...,...,...,...,...,...
321,5,2,1,0.970963,3.375,6.375,3.125,0.0,106500.0
322,3,0,5,0.992383,4.250,6.500,3.250,0.0,34000.0
323,5,1,2,0.998487,4.625,5.250,2.750,1.0,82700.0
324,0,0,8,0.899729,5.250,4.375,5.500,0.0,38962.0


In [49]:
y = df['prediction-label']
X = df.drop(columns = ['prediction-label'])

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [57]:
  
# 1. Create the LogisticRegression model object below and assign to variable 'model'

# YOUR CODE HERE
model = LogisticRegression()

# 2. Fit the model to the training data below

# YOUR CODE HERE
model.fit(X_train, y_train)

# 3. Make predictions on the test data using the predict_proba() method and assign the 
# result to the variable 'probability_predictions' below

# YOUR CODE HERE
probability_predictions = model.predict_proba(X_test)

# print the first 5 probability class predictions
df_print = pd.DataFrame(probability_predictions, columns = ['Class: 0', 'Class: 1'])
print('Class Prediction Probabilities: \n' + df_print[0:5].to_string(index=False))

# 4. Compute the log loss on 'probability_predictions' and save the result to the variable
# 'l_loss' below

# YOUR CODE HERE
l_loss = log_loss(y_test, probability_predictions)
print('Log loss: ' + str(l_loss))


# 5. Make predictions on the test data using the predict() method and assign the result 
# to the variable 'class_label_predictions' below

# YOUR CODE HERE
class_label_predictions = model.predict(X_test)

# print the first 5 class label predictions 
print('Class labels: ' + str(class_label_predictions[0:5]))

# 6.Compute the accuracy score on 'class_label_predictions' and save the result 
# to the variable 'acc_score' below

# YOUR CODE HERE
acc_score = accuracy_score(y_test, class_label_predictions)
print('Accuracy: ' + str(acc_score))

#readability add descriptive labels in output

Class Prediction Probabilities: 
 Class: 0  Class: 1
 0.400377  0.599623
 0.611841  0.388159
 0.371825  0.628175
 0.518613  0.481387
 0.298773  0.701227
Log loss: 0.6517689261512244
Class labels: [1. 0. 1. 0. 1.]
Accuracy: 0.6893203883495146


C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [121]:
!pip install scikit-learn
!pip install torch

DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\matlabengine-24.2-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\matlabengine-24.2-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [200]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Define the LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Define a fully connected (linear) layer for output
        self.fc = nn.Linear(hidden_size, output_size)

        #sigmoid function because this is classification
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # LSTM outputs
        lstm_out, (hn, cn) = self.lstm(x)  # hn is the hidden state from the last LSTM layer
        
        # We take the output from the last time step
        out = self.fc(hn[-1])
        out = self.sigmoid(out)
          
        return out

# Hyperparameters
input_size = 8  # Number of features in each time step (can vary based on your dataset)
hidden_size = 50  # Number of LSTM units in each layer
output_size = 1  # For regression, change this for classification (e.g., number of classes)
num_layers = 1  # Number of LSTM layers
batch_size = 32
seq_length = 20  # Length of each input sequence
num_epochs = 100
learning_rate = 0.001

# Generate some synthetic data for demonstration (e.g., time-series regression)
# Let's generate a random sequence with 1000 samples of length 20
df = pd.read_csv('stock_data.csv')
df = df[df["ticker"]=="AAPL"]
df = df.select_dtypes(include=[np.number]).dropna()

# Convert data into sequences for LSTM input
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data.iloc[i:i+seq_length, 0].values  # Accessing the first column (numerical data)
        y = data.iloc[i+seq_length, 0]  # The value at the next timestep
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Set sequence length
SEQ_LENGTH = 8

# Create sequences
X, y = create_sequences(df, SEQ_LENGTH)


# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)


# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)

# Create DataLoader for batching
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


# Instantiate the model
model = LSTMModel(input_size, hidden_size, output_size, num_layers)

# Loss function and optimizer
criterion = nn.BCELoss()  # Using binary cross entropy loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    
    outputs = model(inputs) # Get model predictions for the current batch
    loss = criterion(outputs, targets)  # Calculate the loss
    
    # Backward pass and optimization
    optimizer.zero_grad()  # Clear previous gradients
    loss.backward()  # Compute gradients
    optimizer.step()  # Update model parameters
    
    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0 and batch_idx == len(train_loader) - 1:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        
          
# After training, you can make predictions like this:
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    test_input = torch.randn(1, seq_length, input_size)  # A single sample
    prediction = model(test_input)
    predicted_class = (prediction >= 0.5).float()  # Classify as 1 if probability >= 0.5, else 0
    print("Prediction:", predicted_class.item())

RuntimeError: all elements of target should be between 0 and 1

In [202]:
df

,Open,High,Low,Close,Adj Close,Volume,neutral-count-finbert,positive-count-finbert,negative-count-finbert,average-confidence-finbert,average-neutral-score-gemini,average-positive-score-gemini,average-negative-score-gemini,prediction-label,number-employees
1,227.139999,228.660004,226.408401,226.960007,226.960007,37608324,2,2,4,0.893756,6.875,5.375,3.500,0.0,164000.0
13,227.139999,228.660004,226.408401,226.960007,226.960007,37608324,1,0,7,0.974607,6.250,2.375,1.750,0.0,164000.0
25,225.054993,225.699997,221.500000,224.229996,224.229996,41415717,3,0,5,0.945336,6.500,6.500,1.375,0.0,164000.0
38,224.630005,225.589996,223.354996,224.229996,224.229996,34988619,2,1,5,0.993764,7.500,4.750,2.750,0.0,164000.0
51,224.009995,226.649994,222.759995,225.119995,225.119995,47686733,4,0,4,0.905029,6.750,6.125,2.125,1.0,164000.0
64,225.070007,228.869995,225.050003,228.220001,228.220001,43401721,3,0,5,0.939148,6.000,5.875,1.750,1.0,164000.0
77,225.919998,226.919998,224.270004,225.000000,225.000000,46862701,2,1,5,0.991967,6.125,2.750,5.250,0.0,164000.0
101,228.789993,230.149994,225.710297,228.520004,228.520004,40355157,4,1,3,0.932328,5.000,5.375,4.625,0.0,164000.0
191,228.059998,230.719894,228.059998,229.869995,229.869995,36963884,5,0,3,0.994476,3.875,7.125,2.875,1.0,164000.0
284,231.460007,233.244995,229.740005,232.869995,232.869995,86333323,2,1,5,0.946358,5.250,4.500,4.125,1.0,164000.0
